In [5]:
from torchkge.utils.datasets import load_wn18rr
from transformers import BertTokenizer, BertModel
# Load dataset
kg = load_wn18rr()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
from nltk.corpus import wordnet as wn
# kg_train.ent2ix
bert2wn = {}

for token, id in tokenizer.vocab.items():
    synsets = wn.synsets(token)
    synset_idx = [kg.ent2ix[synset.name()] for synset in synsets if synset.name() in kg.ent2ix]
    bert2wn[int(id)] = synset_idx


In [7]:
import json

with open('bert2wn.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(bert2wn, indent=4))
# json.dumps(bert2wn)

In [4]:
import torch
from torch import nn
from transformers import BertTokenizer
from models.modeling_bert import BertModel, BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

  0%|          | 0/1000 [00:00<?, ?epoch/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.wordnet_embeddings.transform.0.bias', 'bert.embeddings.wordnet_embeddings.transform.0.weight', 'bert.embeddings.wordnet_embeddings.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to

In [7]:
inputs = tokenizer("I want to [MASK] to school.", return_tensors='pt')
labels = tokenizer("I want to go to school.", return_tensors='pt')['input_ids'].cuda()
for k, v in inputs.items():
    inputs[k] = v.cuda()

model.eval()
model.cuda()

with torch.no_grad():
    outputs = model(**inputs, labels=labels)

In [14]:
print(outputs.loss)
output_ids = torch.argmax(outputs.logits, dim=2)
print(output_ids)
print(tokenizer.batch_decode(output_ids))

tensor(3.3037, device='cuda:0')
tensor([[1012, 1045, 2215, 2000, 2175, 2000, 2082, 1012, 1012]],
       device='cuda:0')
['. i want to go to school..']
